In [9]:
import pandas as pd

df = pd.read_csv("Data/refined-real-estate.csv")
df

,Unnamed: 0,접수연도,자치구명,법정동명,본번,부번,건물명,물건금액(만원),건물면적(㎡),층,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명,계약연도,계약월,계약일
0,0,2025,도봉구,방학동,638,10,한신빌라(638-10),14800,37.66,4,1994,연립다세대,중개거래,서울 도봉구,2025,1,17
1,2,2025,강서구,등촌동,628,13,현대프린스텔,10300,26.16,6,1999,오피스텔,중개거래,서울 강서구,2025,1,17
2,3,2025,관악구,신림동,1639,51,푸리마타운,12700,18.70,8,2014,오피스텔,중개거래,서울 관악구,2025,1,17
3,4,2025,구로구,고척동,339,0,고척파크푸르지오,84000,59.89,9,2009,아파트,중개거래,"서울 구로구, 서울 양천구",2025,1,17
4,5,2025,광진구,중곡동,73,70,욱현하이브(73-70),19800,44.96,3,2009,연립다세대,중개거래,서울 동작구,2025,1,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224571,235664,2022,송파구,거여동,234,0,e편한세상 송파 파크센트럴,69905,59.96,32,0,아파트,신고없음,없음,2018,1,24
224572,235665,2022,강서구,방화동,186,3,마곡미라클,22500,40.11,6,2017,연립다세대,신고없음,없음,2017,7,12
224573,235667,2022,성북구,정릉동,260,10,진안빌라A(260-10),35000,57.95,2,1991,연립다세대,신고없음,없음,2016,6,20
224574,235669,2022,성북구,정릉동,260,10,진안빌라A(260-10),35000,57.95,2,1991,연립다세대,신고없음,없음,2016,4,26


In [10]:
address_names = set()

for district_name, dong_name, main_number, sub_number in zip(
    df["자치구명"], df["법정동명"], df["본번"], df["부번"]
):
    address_name = (
        f"서울특별시 {district_name} {dong_name} {main_number}"
        if sub_number == 0
        else f"서울특별시 {district_name} {dong_name} {main_number}-{sub_number}"
    )
    address_names.add(address_name)
address_names, len(address_names)

({'서울특별시 강동구 명일동 41',
  '서울특별시 양천구 신정동 333',
  '서울특별시 구로구 개봉동 261-10',
  '서울특별시 강동구 길동 320-2',
  '서울특별시 동작구 상도동 264-124',
  '서울특별시 동대문구 용두동 39-860',
  '서울특별시 종로구 동숭동 4-147',
  '서울특별시 송파구 석촌동 175-12',
  '서울특별시 마포구 망원동 435-37',
  '서울특별시 금천구 시흥동 984-16',
  '서울특별시 성북구 정릉동 206-67',
  '서울특별시 영등포구 양평동1가 148-9',
  '서울특별시 광진구 중곡동 85-1',
  '서울특별시 도봉구 방학동 736',
  '서울특별시 강동구 천호동 145-15',
  '서울특별시 도봉구 방학동 608-57',
  '서울특별시 강서구 화곡동 840-3',
  '서울특별시 종로구 창신동 629-62',
  '서울특별시 강남구 역삼동 747-5',
  '서울특별시 은평구 대조동 48-2',
  '서울특별시 강북구 우이동 5-5',
  '서울특별시 도봉구 도봉동 566-15',
  '서울특별시 중랑구 면목동 105-4',
  '서울특별시 강동구 암사동 425-22',
  '서울특별시 성북구 길음동 1285',
  '서울특별시 서대문구 홍은동 277-123',
  '서울특별시 관악구 신림동 1733',
  '서울특별시 노원구 공릉동 758',
  '서울특별시 서대문구 대현동 27-20',
  '서울특별시 금천구 독산동 182-33',
  '서울특별시 서대문구 연희동 100-3',
  '서울특별시 마포구 합정동 386-10',
  '서울특별시 양천구 신월동 71-22',
  '서울특별시 서대문구 남가좌동 5-218',
  '서울특별시 광진구 중곡동 296-27',
  '서울특별시 강서구 화곡동 827-13',
  '서울특별시 은평구 신사동 29-26',
  '서울특별시 용산구 이촌동 426',
  '서울특별시 서대문구 연희동 170-149',
  '서울특별시 중랑구

In [11]:
import requests
import os
from tqdm import tqdm

url = "https://dapi.kakao.com/v2/local/search/address.json"
headers = {"Authorization": f"KakaoAK {os.getenv("KAKAO_API_KEY")}"}
address_to_geo = []
for address_name in tqdm(address_names, desc="Fetching data.."):
    params = {"query": address_name, "analyze_type": "exact"}
    response = requests.get(url, headers=headers, params=params)
    docs = response.json().get("documents")
    if not docs:
        print(f"Error: {response.status_code}, {address_name}")
        continue
    x, y = docs[0].get("x"), docs[0].get("y")
    address_to_geo.append({"address": address_name, "lon": x, "lat": y})

Fetching data..:   2%|▏         | 661/43984 [00:45<43:45, 16.50it/s]  

Error: 200, 서울특별시 동대문구 청량리동 199


Fetching data..:   3%|▎         | 1267/43984 [01:24<44:06, 16.14it/s] 

Error: 200, 서울특별시 강서구 방화동 1171


Fetching data..:   3%|▎         | 1323/43984 [01:27<45:02, 15.79it/s]

...

Fetching data..:  97%|█████████▋| 42512/43984 [45:42<01:40, 14.63it/s]

Error: 200, 서울특별시 강서구 방화동 175-12


Fetching data..:  98%|█████████▊| 43286/43984 [46:33<00:51, 13.46it/s]

Error: 200, 서울특별시 강서구 방화동 168-70


Fetching data..: 100%|██████████| 43984/43984 [47:22<00:00, 15.47it/s]


In [13]:
import csv

with open("Data/address-to-geo.csv", "w", newline="", encoding="utf-8") as file:
    fieldnames = ["address", "lon", "lat"]
    writer = csv.DictWriter(file, fieldnames, lineterminator="\n")
    writer.writeheader()
    writer.writerows(address_to_geo)